In [ ]:
# 패키지 설치
!pip install pickle5 --quiet

In [ ]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pickle5 as pickle
sns.set()

In [ ]:
# 데이터 로드
with open('factor.pkl', 'rb') as f:
    df = pickle.load(f)

In [ ]:
# 시장국면 데이터
regime = df.iloc[:, :5]

# 팩터 데이터
factor = df.iloc[:, 5:]

In [ ]:
# 시장국면 데이터 표준화
scaler = StandardScaler()
rescaled_regime = scaler.fit_transform(regime)
rescaled_regime_df = pd.DataFrame(rescaled_regime, index=regime.index, columns=regime.columns)

In [ ]:
# 표준화 결과 시각화
rescaled_regime_df.plot(figsize=(12, 6))

In [ ]:
# 국면 분석 기준 설정
# 하단 기준선
low_threshold = rescaled_regime_df.quantile(1/3)

# 상단 기준선
high_threshold = rescaled_regime_df.quantile(2/3)

In [ ]:
# 낮은 국면
low_regime = (rescaled_regime_df < low_threshold) * 1

# 높은 국면
high_regime = (rescaled_regime_df > high_threshold) * 1

# 중간 국면
middle_regime = ((rescaled_regime_df >= low_threshold) & (rescaled_regime_df <= high_threshold)) * 1

In [ ]:
low_regime

In [ ]:
factor.cumsum().plot(legend=False)

In [ ]:
# 빈 딕셔너리 생성
low_regime_dict = {}
middle_regime_dict = {}
high_regime_dict = {}

# 국면별 팩터 평균수익률 저장
for rgm in regime.columns:
    low_regime_dict[rgm] = (factor.multiply(low_regime[rgm], axis=0).sum() / np.count_nonzero(np.array(factor.multiply(low_regime[rgm], axis=0)))) * 252
    middle_regime_dict[rgm] = (factor.multiply(middle_regime[rgm], axis=0).sum() / np.count_nonzero(np.array(factor.multiply(middle_regime[rgm], axis=0)))) * 252
    high_regime_dict[rgm] = (factor.multiply(high_regime[rgm], axis=0).sum() / np.count_nonzero(np.array(factor.multiply(high_regime[rgm], axis=0)))) * 252

In [ ]:
# 낮은 국면 데이터프레임
low_regime_df = pd.DataFrame(low_regime_dict)

# 중간 국면 데이터프레임
middle_regime_df = pd.DataFrame(middle_regime_dict)

# 높은 국면 데이터프레임
high_regime_df = pd.DataFrame(high_regime_dict)

In [ ]:
# 전체 국면 분석 종합 딕셔너리
total_regime_dict = {}

for rgm in regime.columns:
    temp_df = pd.concat([low_regime_df[rgm], middle_regime_df[rgm], high_regime_df[rgm]], axis=1, join='inner')
    temp_df.columns = ['low', 'middle', 'high']
    total_regime_dict[rgm] = temp_df

In [ ]:
# 경제성장
sns.heatmap(total_regime_dict['Growth'], cmap='coolwarm')

In [ ]:
# 물가
sns.heatmap(total_regime_dict['Inflation'], cmap='coolwarm')

In [ ]:
# 변동성
sns.heatmap(total_regime_dict['Volatilities'], cmap='coolwarm')

In [ ]:
# 시장 유동성
sns.heatmap(total_regime_dict['Market Liquidity'], cmap='coolwarm')

In [ ]:
# 자금 유동성
sns.heatmap(total_regime_dict['Funding Liquidity'], cmap='coolwarm')